In [12]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
import spacy

creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
245


In [2]:
alertP = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)
new_data=pd.read_sql("""SELECT * FROM hgo_data_032023""",mydb)
alertP1=alertP.copy()
alertP1
alertP1.info()
alertP1.head()

/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10150 entries, 0 to 10149
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_DOENTE            10150 non-null  object
 1   PROCESSO             10150 non-null  object
 2   COD_REFERENCIA       10150 non-null  object
 3   COD_PZ               10150 non-null  object
 4   COD_UNID_SAUDE_PROV  10150 non-null  int64 
 5   UNID_PROV            10150 non-null  object
 6   TIPO_UNID            10150 non-null  object
 7   PROVENIENCIA         10150 non-null  object
 8   COD_CTH_PRIOR        10150 non-null  object
 9   CTH_PRIOR            10150 non-null  object
 10  COD_MOTIVO_RECUSA    10150 non-null  int64 
 11  DES_MOTIVO_RECUSA    10150 non-null  object
 12  COD_ESPECIALIDADE    10150 non-null  int64 
 13  DES_ESPECIALIDADE    10150 non-null  object
 14  agrupadora           10150 non-null  object
 15  OUTRA_ENTIDADE       10150 non-null  int64 
 16  DATA

/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,DATA_MARCACAO,DATA_REALIZACAO,OBSERVACOES,Mês_entrada,Ano_entrada,trata data recusa,resume saída,mês_saida,ano_saida,Texto
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,21/05/2015,12/06/2015,,11,2014,21/05/2015,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an..."
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,,,...,27/04/2012,02/07/2012,,4,2012,23/04/2012,02/07/2012,7,2012,Avaliação neurologica para avaliação da toma d...
2,ADCFD,LVMMVJ,LQHROQHL,,0,,OUTRA,SAM,,,...,26/09/2014,13/01/2015,,9,2014,26/09/2014,13/01/2015,1,2015,cefaleias
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,,...,28/10/2013,03/12/2013,,10,2013,25/10/2013,03/12/2013,12,2013,"Utente do sexo feminino com 30 anos de idade, ..."
4,SEAHEG,LVMNDL,LPHHVHVL,,0,,OUTRA,,,,...,,,Conforme e-mail de 30/11/2015,2,2013,26/02/2013,26/02/2013,2,2013,Sem relatorio clinico


In [3]:
#changing columns with date to a datetime
date_format_alertP1(alertP1)
#change the blank values in proveniencia and priority to unknown
replace_blank(alertP1)
#create a column with the output of a referral and select the refused and accepted ones
alertP1=result(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'


In [4]:
class_area(alertP1)
text_length(alertP1)
referral_steps(alertP1)
speciality(alertP1)
alertP1=unit(alertP1)
alertP1=alertP1.sort_values('DATA_RECEPCAO')
alertP1

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/features.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['step'][alertP1['step']>=3]='3+'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/features.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['unit'][alertP1['TIPO_UNID']!='CS/USF']='HOSP'


,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,resume saída,mês_saida,ano_saida,Texto,result,area,text_length,step,speciality_type,unit
9915,SEAFHB,BBLVVLDC,LOHHHLOV,SFGHADG,3150305,UCSP RAINHA D. LEONOR,CS/USF,CTH,,unknown,...,30/03/2012,3,2012,,1,inside area,0,1,Other specialities,UCSP
6944,ZDGGGA,BMJNMVVJ,LOHHHHRS,,0,,OUTRA,unknown,,unknown,...,01/08/2012,8,2012,,1,outside area,0,1,General Neurology,HOSP
6542,ZZZBGG,BMJIBILB,LOHHHOHO,SSBSEGA,3150301,FRANCISCO X NORONHA,CS/USF,CTH,,unknown,...,10/01/2012,1,2012,,0,outside area,0,1,General Neurology,CS
3067,DSHCGG,BMIBJIE,LOHHHOUQ,SFAAECF,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,12/03/2012,3,2012,,1,inside area,0,1,Other specialities,USF
2078,AESBSC,NINBJJC,LOHHHPTO,,0,,OUTRA,unknown,,unknown,...,22/01/2013,1,2013,,1,outside area,0,1,General Neurology,HOSP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7353,ZSGCCF,BMJCJLIJ,LSHVLHUO,ZEFHCECF,3150672,USF FEIJO-CS COVA DA PIEDADE,CS/USF,CTH,3,Normal,...,20/03/2017,3,2017,,1,inside area,0,2,General Neurology,CS
4044,CDABCC,VLNVILNB,LSHVLLUO,ZEFZDDBB,3150371,USF S JOAO DO PRAGAL-CS ALMADA,CS/USF,CTH,3,Normal,...,21/03/2017,3,2017,,1,inside area,0,1,General Neurology,USF
6424,FCDEF,BIJIMMIJ,LSHVLOQU,,0,,OUTRA,SAM,3,Normal,...,01/03/2018,3,2018,,1,outside area,0,2,Other specialities,HOSP
9425,EBESGE,BDVDJJIB,LSHVLLVV,,0,,OUTRA,SAM,,unknown,...,04/09/2017,9,2017,,1,outside area,0,1,Other specialities,HOSP


In [5]:
alertP1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9374 entries, 9915 to 5513
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_DOENTE            9374 non-null   object        
 1   PROCESSO             9374 non-null   object        
 2   COD_REFERENCIA       9374 non-null   object        
 3   COD_PZ               9374 non-null   object        
 4   COD_UNID_SAUDE_PROV  9374 non-null   int64         
 5   UNID_PROV            9374 non-null   object        
 6   TIPO_UNID            9374 non-null   object        
 7   PROVENIENCIA         9374 non-null   object        
 8   COD_CTH_PRIOR        9374 non-null   object        
 9   CTH_PRIOR            9374 non-null   object        
 10  COD_MOTIVO_RECUSA    9374 non-null   int64         
 11  DES_MOTIVO_RECUSA    9374 non-null   object        
 12  COD_ESPECIALIDADE    9374 non-null   int64         
 13  DES_ESPECIALIDADE    9374 non-

In [6]:
twin = alertP[(alertP["COD_MOTIVO_RECUSA"]==18) | (alertP["COD_MOTIVO_RECUSA"]==19)]
twin.head()


,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,DATA_MARCACAO,DATA_REALIZACAO,OBSERVACOES,Mês_entrada,Ano_entrada,trata data recusa,resume saída,mês_saida,ano_saida,Texto
11,SSDCCC,LVNICL,LOHRRVHV,,0,,OUTRA,,,,...,,,realizou consulta com DR Joao Proenca a 13.05....,12,2012,28/12/2012,28/12/2012,12,2012,a
20,SSDZBS,LVEVCB,LPHOQRHO,,0,,OUTRA,,,,...,,,,5,2013,30/05/2013,30/05/2013,5,2013,a
28,SEBAGZ,LVCJVE,LSHTUSUR,,0,,OUTRA,SAM,,,...,,,,11,2016,10/11/2016,10/11/2016,11,2016,"suspeita de miastenia gravis. Disfagia, diplop..."
32,EBBHZA,LVCVDN,LOHLLRPS,,0,,OUTRA,,,,...,,,,3,2012,08/03/2012,08/03/2012,3,2012,O Doente ja teve Consulta com outra referencia
57,SSCADA,LVDCMM,LPHQTOOH,,0,,OUTRA,,,,...,,,,10,2013,19/06/2014,19/06/2014,6,2014,Referencia Duplicada


In [7]:
merg = pd.merge(twin, alertP1, on='ID_DOENTE', how='left')
merg.sort_values(by=["ID_DOENTE"])

,ID_DOENTE,PROCESSO_x,COD_REFERENCIA_x,COD_PZ_x,COD_UNID_SAUDE_PROV_x,UNID_PROV_x,TIPO_UNID_x,PROVENIENCIA_x,COD_CTH_PRIOR_x,CTH_PRIOR_x,...,resume saída_y,mês_saida_y,ano_saida_y,Texto_y,result,area,text_length,step,speciality_type,unit
64,AAHGS,VLNBLDE,LPHPHRVP,,0,,OUTRA,,,,...,29/01/2015,1,2015,"F, 68a Tonturas/instabilidade Tremor de rep...",1,outside area,65.0,1,General Neurology,HOSP
304,ABDZSH,VLNVBVVN,LQHLQQPV,,0,,OUTRA,SAM,,,...,31/03/2014,3,2014,,0,outside area,0.0,1,General Neurology,HOSP
303,ABDZSH,VLNVBVVN,LQHLQQPT,,0,,OUTRA,SAM,,,...,31/03/2014,3,2014,,0,outside area,0.0,1,General Neurology,HOSP
186,ABEEHA,EIDIJCI,LOHPSTVU,,0,,OUTRA,,,,...,09/03/2015,3,2015,,1,outside area,0.0,1,General Neurology,HOSP
187,ABGZES,CIDCMBJ,LRHSSLUU,,0,,OUTRA,SAM,,,...,02/03/2016,3,2016,,1,inside area,0.0,1,General Neurology,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,ZZGEBC,BMJMNMME,LPHQRHVV,,0,,OUTRA,,,,...,25/09/2013,9,2013,,0,outside area,0.0,1,General Neurology,HOSP
458,ZZGEBC,BMJMNMME,LPHQRHVV,,0,,OUTRA,,,,...,10/03/2016,3,2016,,1,inside area,0.0,2,General Neurology,USF
456,ZZGZFG,BMJMNLCI,LQHLSQUH,BDEDCBD,3150572,USF MONTE DA CAPARICA,CS/USF,CTH,3,Normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630,ZZHEBH,BCVEVVCI,LRHLOVUV,,0,,OUTRA,SAM,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Column1    362 non-null    int64 
 1   COD_PZ     362 non-null    object
 2   ID_DOENTE  362 non-null    object
 3   text       362 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.4+ KB


,Column1,COD_PZ,ID_DOENTE_x,text,ID_DOENTE_y,PROCESSO,COD_REFERENCIA,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,...,DATA_MARCACAO,DATA_REALIZACAO,OBSERVACOES,Mês_entrada,Ano_entrada,trata data recusa,resume saída,mês_saida,ano_saida,Texto
0,0,GGGGAGG,ZFFBAE,"epilepsia, crises 1x/semana sob medicação Paul...",ZFFBAE,BMJCNLJV,LRHOQQLT,3150672.0,USF FEIJO-CS COVA DA PIEDADE,CS/USF,...,21/08/2015,09/09/2015,,4.0,2015.0,20/08/2015,09/09/2015,9,2015,
1,1,GGSFCSA,ZDGCAF,Necessidade de avaliação especializada António...,ZDGCAF,VNNEBBDM,LRHOLTQO,3150305.0,UCSP RAINHA D. LEONOR,CS/USF,...,20/08/2015,03/09/2015,,4.0,2015.0,20/08/2015,03/09/2015,9,2015,
2,2,GBSSZBG,ZFZFGB,Solicito observação na consulta de Neurologia ...,ZFZFGB,BMJCVCLD,LRHLTHVO,3150572.0,USF MONTE DA CAPARICA,CS/USF,...,20/07/2015,22/09/2015,,3.0,2015.0,13/07/2015,22/09/2015,9,2015,
3,3,GBECBGB,DHGHFZ,Cefaleias Alterações cognitivasPatrícia Costa ...,DHGHFZ,BNJBJBMM,LRHLSRHU,3150571.0,USF SOBREDA-CS COSTA CAPARICA,CS/USF,...,20/07/2015,31/08/2015,,3.0,2015.0,13/07/2015,31/08/2015,8,2015,
4,4,GAZGHZD,ECEHHZ,"Polineuropatia Isabel Ferreira, Drª. (Medicina...",ECEHHZ,BBVBVVVE,LRHHTSQV,3150672.0,USF FEIJO-CS COVA DA PIEDADE,CS/USF,...,21/07/2015,21/09/2015,,2.0,2015.0,09/07/2015,21/09/2015,9,2015,
5,5,GSGFEFD+B23,ZZZZZD,"Cefaleias Paula Costa,Drª (Medicina Geral e Fa...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,GSBAFAZ,BSFZHE,"Tremor. Parkinsonismo?Paula Costa, Drª. (Medic...",BSFZHE,VMNMBCMJ,LRHHHLOH,3150371.0,USF S JOAO DO PRAGAL-CS ALMADA,CS/USF,...,13/07/2015,03/08/2015,,1.0,2015.0,09/07/2015,03/08/2015,8,2015,
7,7,GSHAGAZ,EZCCH,"lesao hipodensa profunda do hemisfero dto, ca...",EZCCH,BIJVEJJE,LQHSVLQS,3150672.0,USF FEIJO-CS COVA DA PIEDADE,CS/USF,...,02/04/2015,27/04/2015,,12.0,2014.0,02/04/2015,27/04/2015,4,2015,
8,8,GECDCAE,BBDEEC,"NACIDO SANTO TOME, CHEGOU 30/10/2012 NAO TEM B...",BBDEEC,VNNEDLDL,LQHSTOLV,3152401.0,UCSP CORROIOS,CS/USF,...,11/06/2015,10/07/2015,,12.0,2014.0,11/06/2015,10/07/2015,7,2015,
9,9,GESGHBA,SHABAS,"Síndrome demencial Carolina Costa , Drª. (Medi...",SHABAS,VNNEDLDJ,LQHSSPLV,3150571.0,USF SOBREDA-CS COSTA CAPARICA,CS/USF,...,11/06/2015,06/07/2015,,11.0,2014.0,11/06/2015,06/07/2015,7,2015,


,Column1,COD_PZ,ID_DOENTE_x,text,ID_DOENTE_y,PROCESSO,COD_REFERENCIA,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,...,DATA_MARCACAO,DATA_REALIZACAO,OBSERVACOES,Mês_entrada,Ano_entrada,trata data recusa,resume saída,mês_saida,ano_saida,Texto
0,0,GGGGAGG,ZFFBAE,"epilepsia, crises 1x/semana sob medicação Paul...",ZFFBAE,BMJCNLJV,LRHOQQLT,3150672.0,USF FEIJO-CS COVA DA PIEDADE,CS/USF,...,21/08/2015,09/09/2015,,4.0,2015.0,20/08/2015,09/09/2015,9,2015,
1,1,GGSFCSA,ZDGCAF,Necessidade de avaliação especializada António...,ZDGCAF,VNNEBBDM,LRHOLTQO,3150305.0,UCSP RAINHA D. LEONOR,CS/USF,...,20/08/2015,03/09/2015,,4.0,2015.0,20/08/2015,03/09/2015,9,2015,
2,2,GBSSZBG,ZFZFGB,Solicito observação na consulta de Neurologia ...,ZFZFGB,BMJCVCLD,LRHLTHVO,3150572.0,USF MONTE DA CAPARICA,CS/USF,...,20/07/2015,22/09/2015,,3.0,2015.0,13/07/2015,22/09/2015,9,2015,
3,3,GBECBGB,DHGHFZ,Cefaleias Alterações cognitivasPatrícia Costa ...,DHGHFZ,BNJBJBMM,LRHLSRHU,3150571.0,USF SOBREDA-CS COSTA CAPARICA,CS/USF,...,20/07/2015,31/08/2015,,3.0,2015.0,13/07/2015,31/08/2015,8,2015,
4,4,GAZGHZD,ECEHHZ,"Polineuropatia Isabel Ferreira, Drª. (Medicina...",ECEHHZ,BBVBVVVE,LRHHTSQV,3150672.0,USF FEIJO-CS COVA DA PIEDADE,CS/USF,...,21/07/2015,21/09/2015,,2.0,2015.0,09/07/2015,21/09/2015,9,2015,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,357,ZDEHEZZE,ESSAA,"Parkinsonismo?? Joana Costa , Drª. (Medicina G...",ESSAA,LIJVJ,LSHPSHSP,3150671.0,USF COVA DA PIEDADE-CS C PIEDA,CS/USF,...,07/07/2016,07/09/2016,,5.0,2016.0,06/07/2016,07/09/2016,9,2016,tremor das mãos com com cerca de um ano de evo...
358,358,ZDDGZAGD,AZHBBD,Homem de 76 anos com perturbações da marcha e ...,AZHBBD,MILBDLL,LSHPRLHT,3151574.0,USF CSI SEIXAL,CS/USF,...,29/07/2016,09/08/2016,,5.0,2016.0,06/07/2016,09/08/2016,8,2016,Homem de 76 anos com perturbações da marcha e ...
359,359,ZDDEAEZF,FDCCBE,80 anos com hemiparesia espástica à esq. após ...,FDCCBE,LLCLLBB,LSHPOQHP,3150371.0,USF S JOAO DO PRAGAL-CS ALMADA,CS/USF,...,29/06/2016,29/08/2016,,5.0,2016.0,16/06/2016,29/08/2016,8,2016,80 anos com hemiparesia espástica à esq. após ...
360,360,ZDZCZCDF,SCZZHH,Dem~encia no contexto de Parkinson? Maria Cost...,SCZZHH,LIMCJV,LSHOVQSR,3150671.0,USF COVA DA PIEDADE-CS C PIEDA,CS/USF,...,29/06/2016,26/08/2016,,5.0,2016.0,20/05/2016,26/08/2016,8,2016,"Referenciado a Neurologia em 2014, consulta ma..."


NameError: name 'matched_large' is not defined